In [1]:
import pandas as pd
import seaborn as sns

In [4]:
import xgboost

In [3]:
import pickle

# GET REPORT_DATE

In [5]:
REPORT_DATE = '01-06-2023'

## GET FORMATTED AND SCORE

In [5]:
df = pd.read_parquet(f'../data/{REPORT_DATE}/FORMATTED_FEATURE_DATA')

In [24]:
len(df)

16593962

In [6]:
df1 = pd.read_parquet(f'../out/SCORE_{REPORT_DATE}')
df1['CUSTOMER_CDE'] = df1['CUSTOMER_CDE'].astype(int)

In [25]:
len(df1)

16593962

In [7]:
df.columns

Index(['CASA_HOLD', 'CARD_CREDIT_HOLD', 'EB_SACOMPAY_HOLD', 'EB_MBIB_HOLD',
       'LOR', 'CREDIT_SCORE', 'CASA_BAL_SUM_NOW',
       'CASA_DAY_SINCE_LAST_TXN_CT_36M', 'CARD_CREDIT_MAX_LIMIT',
       'CARD_CREDIT_SUM_BAL_NOW', 'EB_SACOMPAY_DAY_SINCE_LTST_LOGIN',
       'EB_SACOMPAY_DAY_SINCE_LTST_TXN', 'EB_MBIB_DAY_SINCE_ACTIVE',
       'LIFE_STG', 'AREA'],
      dtype='object')

In [8]:
df1.columns

Index(['CUSTOMER_CDE', 'SCORE'], dtype='object')

In [9]:
# Fix index
df.index = df1['CUSTOMER_CDE']
df = df.reset_index()

In [10]:
# df = df.sample(n=100000).reset_index(drop=True)

In [12]:
df['CUSTOMER_CDE'].dtype, df1['CUSTOMER_CDE'].dtype

(dtype('int64'), dtype('int64'))

## MERGE together

In [13]:
df2 = df1[['CUSTOMER_CDE', 'SCORE']].merge(df,how='inner', left_on='CUSTOMER_CDE', right_on='CUSTOMER_CDE')

In [16]:
df2.head()

,CUSTOMER_CDE,SCORE,CASA_HOLD,CARD_CREDIT_HOLD,EB_SACOMPAY_HOLD,EB_MBIB_HOLD,LOR,CREDIT_SCORE,CASA_BAL_SUM_NOW,CASA_DAY_SINCE_LAST_TXN_CT_36M,CARD_CREDIT_MAX_LIMIT,CARD_CREDIT_SUM_BAL_NOW,EB_SACOMPAY_DAY_SINCE_LTST_LOGIN,EB_SACOMPAY_DAY_SINCE_LTST_TXN,EB_MBIB_DAY_SINCE_ACTIVE,LIFE_STG,AREA
0,14557591,0.994378,NaN,NaN,NaN,NaN,2557.0,NaN,NaN,1095,NaN,NaN,NaN,NaN,NaN,Lập gia đình,VN0010005
1,1071696,0.991121,NaN,NaN,NaN,NaN,2771.0,NaN,NaN,1095,NaN,NaN,NaN,NaN,NaN,Lập gia đình,VN0010003
2,8356085,0.991121,1.0,NaN,NaN,NaN,3815.0,NaN,11291.0,1,NaN,NaN,NaN,NaN,NaN,Cuối sự nghiệp,VN0010005
3,14741827,0.991019,1.0,NaN,1.0,NaN,3752.0,NaN,118121.0,1,NaN,NaN,NaN,21.0,NaN,Thiết lập tài sản,VN0010006
4,6742801,0.990698,1.0,NaN,NaN,NaN,1787.0,NaN,54721.0,1,NaN,NaN,NaN,NaN,NaN,Thiết lập tài sản,VN0010004


In [14]:
len(df2)

16593962

## GET THRESHOLD OPTIONS of each PRODUCTS

In [19]:
for s in ['CASA_HOLD', 'CARD_CREDIT_HOLD', 'EB_SACOMPAY_HOLD', 'EB_MBIB_HOLD']:
    df3 = df2[df2[s]==1]
    print(f'{s}\n\tTotal customers gt 0.5\t:', len(df3[df3['SCORE']>=0.5]))
    print('\tThreshold at 100k\t:', df3.iloc[100000]['SCORE'])

CASA_HOLD
	Total customers gt 0.5	: 28247
	Threshold at 100k	: 0.14381897
CARD_CREDIT_HOLD
	Total customers gt 0.5	: 3306
	Threshold at 100k	: 0.021131735
EB_SACOMPAY_HOLD
	Total customers gt 0.5	: 9968
	Threshold at 100k	: 0.05019249
EB_MBIB_HOLD
	Total customers gt 0.5	: 9143
	Threshold at 100k	: 0.04605497


## GET THRESHOLD at 100k

In [23]:
df2.iloc[100000]['SCORE']

0.32027942

In [ ]:
TO_DATE('01-06-23','DD-MM-YY')